In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import random

# Load MedQA textbooks
Or other passages

In [5]:
source = "/scratch/s190619/Data_etc/MedQA/textbooks/combined_text_books.txt"

textbooks = []
f = open(source, "r")
for x in f:
    textbooks.append(x)
f.close()

TB = [x.replace("\n","") for x in textbooks if x != "\n"]
TB_l = [x.lower() for x in TB]

In [10]:
print(TB[1000])
print("")
print(TB[3090])
print("")
print(TB[4444])

The arrangement of vessels and nerves that supply the thoracic wall reflects the segmental organization of the wall. Arteries to the wall arise from two sources: the thoracic aorta, which is in the posterior mediastinum, and a pair of vessels, the internal thoracic arteries, which run along the deep aspect of the anterior thoracic wall on either side of the sternum.

At first glance, it is difficult to appreciate why the psoas muscle sheath is of greater importance than any other muscle sheath. The psoas muscle and its sheath arise not only from the lumbar vertebrae but also from the intervertebral discs between each vertebra. This disc origin is of critical importance. In certain types of infection, the intervertebral disc is preferentially affected (e.g., tuberculosis and salmonella discitis). As the infection of the disc develops, the infection spreads anteriorly and anterolaterally. In the anterolateral position, the infection passes into the psoas muscle sheath, and spreads within

# Load queries

In [9]:
queries = pd.read_csv("/scratch/s190619/MSMARCO/MedQA_US/MedQAQueries_disorders_full.csv",sep=",")
queries.head(5)

,Unnamed: 0,qid,query,answer,options,meta_info,Disorder,CUI,TUI,short_category,long_category,description,cui_METAMAP
0,0,3,A pulmonary autopsy specimen from a 58-year-ol...,Thromboembolism,"{'A': 'Thromboembolism', 'B': 'Pulmonary ische...",step1,True,C0040038,T046,DISO,Disorders,Pathologic Function,['C0040038']
1,1,4,A 20-year-old woman presents with menorrhagia ...,Von Willebrand disease,"{'A': 'Factor V Leiden', 'B': 'Hemophilia A', ...",step1,True,C0042974,T047,DISO,Disorders,Disease or Syndrome,['C0042974']
2,2,5,A 40-year-old zookeeper presents to the emerge...,Scorpion sting,"{'A': 'Aspirin', 'B': 'Oral contraceptive pill...",step1,True,C0238417,T037,DISO,Disorders,Injury or Poisoning,['C0238417']
3,2,5,A 40-year-old zookeeper presents to the emerge...,Scorpion sting,"{'A': 'Aspirin', 'B': 'Oral contraceptive pill...",step1,True,C0261663,T037,DISO,Disorders,Injury or Poisoning,['C0238417']
4,3,21,An investigator is studying the function of th...,Anorexia,"{'A': 'Hypothermia', 'B': 'Hyperthermia', 'C':...",step1,True,C0003123,T047,DISO,Disorders,Disease or Syndrome,"['C0003123', 'C1971624']"


# Create triples

In [43]:
def filter_textbook_by_answers(queries, passages):
    
    qa_pairs = queries[["query","answer"]]
    number_of_unrelated_passages = 10
    
    relevant_passages = []
    unrelevant_passages = []
    for q,a in tqdm(qa_pairs.to_numpy()):
        a_l = a.lower()
        un_pa = []
        try: 
            int(a_l)
            relevant_passages.append([])
            unrelevant_passages.append([])
        except: 
            if len(a_l) >= 4: #The answer to some questions is simply A, B, C, D, E, or some number
                relevant_passages.append([x.replace("\t"," ") for x in passages if a_l in x])
                
                # Get unrelevant passages
                k=0
                while k < number_of_unrelated_passages:
                    ind = random.randint(0,len(passages))
                    if a_l.lower() not in passages[ind] and len(passages[ind]) > 250 and len(passages[ind]) < 2000:
                        un_pa.append(passages[ind].replace("\t"," "))
                        k += 1
                unrelevant_passages.append(un_pa) 
                
            else:
                relevant_passages.append([])
                unrelevant_passages.append([])
    
    qa_pairs["relevant_passages"] = relevant_passages
    qa_pairs["unrelevant_passages"] = unrelevant_passages
    return qa_pairs

In [44]:
%%time
qa_2 = filter_textbook_by_answers(queries, TB_l)

  0%|          | 0/2616 [00:00<?, ?it/s]

CPU times: user 4min 8s, sys: 5.31 s, total: 4min 13s
Wall time: 4min 12s


/home/s190619/.conda/envs/colbert-v0.2/lib/python3.7/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/s190619/.conda/envs/colbert-v0.2/lib/python3.7/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [45]:
qa_2 = qa_2.drop_duplicates(subset="query").reset_index().drop("index", axis=1)
a = len([x for x in qa_2["relevant_passages"] if x==[]])
b = len(qa_2["relevant_passages"])
print("Percentage of questions covered: %.2f percent" % ((1-a/b)*100))
qa_2.head()

Percentage of questions covered: 93.60 percent


,query,answer,relevant_passages,unrelevant_passages
0,A pulmonary autopsy specimen from a 58-year-ol...,Thromboembolism,"[direct thrombin bivalirudin, argatroban, dabi...","[from a clinical perspective, syncope is essen..."
1,A 20-year-old woman presents with menorrhagia ...,Von Willebrand disease,[adhesion of platelets to exposed collagen at ...,[the ovary responds to fsh and lh in a defined...
2,A 40-year-old zookeeper presents to the emerge...,Scorpion sting,"[causes: idiopathic, gallstones, ethanol, trau...",[the differential diagnosis for early clinical...
3,An investigator is studying the function of th...,Anorexia,[kwashiorkor: kwashiorkor occurs when protein ...,[since portion control is one of the most diff...
4,A 42-year-old woman is in the hospital recover...,Delirium tremens,[delirium is a clinical diagnosis that is made...,[antivirals: [p] decreased absorption of antiv...


### BM25 get most relevant passages

In [46]:
from rank_bm25 import BM25Okapi

In [47]:
def BM25_relevant_passages(qap):
    #qap = question answer passages
    max_doc_len = 2000 # Some documents are really long, we do not want those
#     max_tokens = 180
    qs = []
    ans = []
    rel_pas = []
    unrel_pas = []
    
    for index in tqdm(range(qap.shape[0])):
    
        query = qap["query"].iloc[index].lower()
        answer = qap["answer"].iloc[index]
        corpus = [doc for doc in qap["relevant_passages"].iloc[index] if len(doc)<max_doc_len]
        
        if len(corpus) > 0:
            tokenized_corpus = [doc.lower().split(" ") for doc in corpus]
            bm25 = BM25Okapi(tokenized_corpus)
            tokenized_query = query.split(" ")
            doc_scores = bm25.get_scores(tokenized_query)
            N = min(len(doc_scores[np.where(doc_scores>doc_scores.max()/2)]),10)
            relevant_passages = bm25.get_top_n(tokenized_query, corpus, n=N)
            
            #tokenize_passages = [doc.lower().split(" ")[:max_tokens] for doc in relevant_passages]
            #relevant_passages = [" ".join([x for x in doc]) for doc in tokenized_passages]
            
            qs.append(query)
            ans.append(answer)
            rel_pas.append(relevant_passages)
            unrel_pas.append(random.sample(qap["unrelevant_passages"].iloc[index],len(relevant_passages)))
        
    return pd.DataFrame(list(zip(qs,ans,rel_pas,unrel_pas)), columns = ["query","answer","relevant_passages","unrelevant_passages"])

In [48]:
qa_2_bm25 = BM25_relevant_passages(qap = qa_2)
qa_2_bm25.head(1)

  0%|          | 0/2281 [00:00<?, ?it/s]

,query,answer,relevant_passages,unrelevant_passages
0,a pulmonary autopsy specimen from a 58-year-ol...,Thromboembolism,[the patient has significant symptoms during r...,[primary respiratory disturbances (primary cha...


In [49]:
qs = []
ans = []
rel_pas = []
unrel_pas = []
for i in tqdm(range(qa_2_bm25.shape[0])):
    q = qa_2_bm25.loc[i,"query"]
    a = qa_2_bm25.loc[i,"answer"]
    tmp_rel_pas = qa_2_bm25.loc[i,"relevant_passages"]
    tmp_unrel_pas = qa_2_bm25.loc[i,"unrelevant_passages"]
    for i in range(len(tmp_rel_pas)):
        qs.append(q)
        ans.append(a)
        rel_pas.append(tmp_rel_pas[i])
        unrel_pas.append(tmp_unrel_pas[i])
        
qa_long = pd.DataFrame(list(zip(qs,ans,rel_pas,unrel_pas)), columns = ["query","answer","relevant_passage","unrelevant_passage"])
qa_long = qa_long.sample(frac=1).reset_index(drop=True)

  0%|          | 0/2121 [00:00<?, ?it/s]

In [50]:
print("Shape: ", qa_long.shape)
qa_long.head()

Shape:  (16645, 4)


,query,answer,relevant_passage,unrelevant_passage
0,a 35-year-old woman comes to the physician bec...,Venous thrombosis,clinical manifestations patients with cancer w...,pure alexia without agraphia this is the visua...
1,a 16-year-old girl comes to the physician beca...,Bulimia nervosa,the most remarkable finding on physical examin...,it must be remarked that the degree of disabil...
2,a 25-year-old woman is brought to the physicia...,Akathisia,rage reactions with continuous violent activit...,regulatory t (treg) cells constitute a populat...
3,a 22-year-old woman is in her last few months ...,Suppression,in patients in whom the diagnosis of cushing’s...,the establishment of hiv as the causative agen...
4,a 7-year-old girl is brought to the physician ...,Type III hypersensitivity,serum sickness is a type iii hypersensitivity ...,"after penetrating the zona pellucida, the sper..."


### Save the results

In [51]:
file_name = "MedQA_textbook_triples_w_answer2.csv"
qa_long.to_csv(file_name, sep=",")

In [52]:
file_name = "MedQA_textbook_triples2.tsv"
qa_triples = qa_long[["query","relevant_passage","unrelevant_passage"]]
qa_triples.to_csv(file_name, sep="\t",index=False,header=False)

# Have a look at the triples

In [5]:
qa_example = pd.read_csv("MedQA_textbook_triples_w_answer2.csv").drop("Unnamed: 0",axis=1)
print(qa_example.shape)
qa_example.head()

(16645, 4)


,query,answer,relevant_passage,unrelevant_passage
0,a 35-year-old woman comes to the physician bec...,Venous thrombosis,clinical manifestations patients with cancer w...,pure alexia without agraphia this is the visua...
1,a 16-year-old girl comes to the physician beca...,Bulimia nervosa,the most remarkable finding on physical examin...,it must be remarked that the degree of disabil...
2,a 25-year-old woman is brought to the physicia...,Akathisia,rage reactions with continuous violent activit...,regulatory t (treg) cells constitute a populat...
3,a 22-year-old woman is in her last few months ...,Suppression,in patients in whom the diagnosis of cushing’s...,the establishment of hiv as the causative agen...
4,a 7-year-old girl is brought to the physician ...,Type III hypersensitivity,serum sickness is a type iii hypersensitivity ...,"after penetrating the zona pellucida, the sper..."


In [4]:
def get_examples(qa, random_sample = True, num = 1, spec_index = [], spec_answer = ""):
    
    if spec_answer != "":
        qa = qa[qa["answer"]==spec_answer].reset_index(drop=True)
    
    if spec_index != []:
        for i in spec_index:
            print("Query:\n%s"%(qa["query"].iloc[i]))
            print("\nAnswer:\n%s"%(qa["answer"].iloc[i]))
            print("\nRelevant passage:\n%s"%(qa["relevant_passage"].iloc[i]))
            print("\nUnrelevant passage:\n%s"%(qa["unrelevant_passage"].iloc[i]))
            print("\n++++++++++++++++++++++++++++++++++++++++++++++++++\n")
            
    else:
        for i in range(num):
            if random_sample == True:
                ind = random.randint(0,qa.shape[0]-1)
            else:
                ind = i
            print("Query:\n%s"%(qa["query"].iloc[ind]))
            print("\nAnswer:\n%s"%(qa["answer"].iloc[ind]))
            print("\nRelevant passage:\n%s"%(qa["relevant_passage"].iloc[ind]))
            print("\nUnrelevant passage:\n%s"%(qa["unrelevant_passage"].iloc[ind]))
            print("\n++++++++++++++++++++++++++++++++++++++++++++++++++\n")

In [14]:
index_list = []
get_examples(qa_example, random_sample = True, num = 5, spec_index = index_list, spec_answer = "Thromboembolism")

Query:
a pulmonary autopsy specimen from a 58-year-old woman who died of acute hypoxic respiratory failure was examined. she had recently undergone surgery for a fractured femur 3 months ago. initial hospital course was uncomplicated, and she was discharged to a rehab facility in good health. shortly after discharge home from rehab, she developed sudden shortness of breath and had cardiac arrest. resuscitation was unsuccessful. on histological examination of lung tissue, fibrous connective tissue around the lumen of the pulmonary artery is observed. which of the following is the most likely pathogenesis for the present findings?

Answer:
Thromboembolism

Relevant passage:
nuclear imaging depends on the selective uptake of various compounds by organs of the body. in thoracic imaging, these compounds are concentrated by one of three mechanisms: blood pool or compartmentalization (e.g., within the heart), physiologic incorporation (e.g., bone or thyroid) and capillary blockage (e.g., lung

In [20]:
max_doc_len = 2000
index = 0
corpus = [doc for doc in qa_2["relevant_passages"].iloc[index] if len(doc)<max_doc_len]
tokenized_corpus = [doc.lower().split(" ")[:3] for doc in corpus]

In [27]:
[" ".join([x for x in doc]) for doc in tokenized_corpus]

['direct thrombin bivalirudin,',
 'clinical use chronic',
 'heparin “bridging”: heparin',
 'contraindications to hrt',
 'oral contraceptive pills',
 'acute tubular necrosis',
 'the combination estrogen–progestin',
 'venous thrombosis women',
 'oc, oral contraceptive;',
 'there is no',
 'lidegaard o, lokkegaard',
 'farmer rdt, preston',
 'heinemann la, dinger',
 'lidegaard o. oral',
 'jick ss, hagberg',
 'because pulmonary embolism',
 'previous surgical procedures,',
 'a family history',
 'the risks and',
 'table 22.11 thromboembolism',
 'age >60 and',
 'the causal factors',
 'graduated compression stockings',
 'the use of',
 'because pulmonary embolism',
 'unfractionated heparin after',
 'low-molecular-weight heparin two',
 'the dosages used',
 'electrocardiogram with large',
 'clarke-pearson dl, jelovsek',
 'jorgensen ln, wille-jorgensen',
 'clarke-pearson dl, synan',
 'clarke-pearson dl, creasman',
 'van dongen cj,',
 'it is important',
 'kovac m, mitic',
 'bates sm, greer',
 'leperc

In [26]:
[x for x in tokenized_corpus[0]]

['direct', 'thrombin', 'bivalirudin,']